In [1]:
# Zelle 1: Imports und Basis-Setup

from groq import Groq
from rdflib import Graph
from rdflib.namespace import RDFS
import json
import textwrap

from groq import Groq

# Pfad zu deiner Datei
api_key_path = r"C:\Users\Alexander Verkhov\Desktop\APIKey_Groq.txt"

# API-Key aus Datei lesen
with open(api_key_path, "r", encoding="utf-8") as f:
    groq_api_key = f.read().strip()  # .strip() entfernt Zeilenumbrüche

# Client mit Key initialisieren
client = Groq(api_key=groq_api_key)

# Testaufruf
resp = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "Hallo, sprichst du auch deutsch?"}],
    max_completion_tokens=500,
)

print(resp.choices[0].message.content)
# Wissensgraph laden (Pfad / Format an dein Projekt anpassen!)
KG_PATH = r"D:\MA_Python_Agent\MSRGuard\KGs\ParamDiag_Agent_filled2.ttl" 
KG_FORMAT = "turtle"               # z.B. 'turtle', 'xml', 'nt', ...

g = Graph()
g.parse(KG_PATH, format=KG_FORMAT)
print(f"Graph geladen mit {len(g)} Tripeln.")


Ja, ich kann Deutsch sprechen. Wie kann ich dir heute helfen? Hast du eine bestimmte Frage oder ein bestimmtes Thema, das du besprechen möchtest?
Graph geladen mit 5253 Tripeln.


In [2]:
# Zelle 2: Groq Helper-Funktion

def call_groq(messages, model="llama-3.1-8b-instant", temperature=0.0, max_tokens=512):
    """
    Einfacher Wrapper für Groq-ChatCompletion (nicht-streamend).
    messages: Liste von {role: "system"/"user"/"assistant", content: "..."}
    """
    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_completion_tokens=max_tokens,
        top_p=1,
        stream=False,
    )
    return resp.choices[0].message.content.strip()


In [3]:
results=g.query("""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX ag:<http://www.semanticweb.org/AgentProgramParams/>
SELECT * WHERE { 

?v a ag:class_Variable.
ag:Program_HRL_SkillSet ag:op_usesVariable ?v.
?v ag:op_isMappedToProgram ?prog.
?prog ag:dp_hasProgramCode ?code.
}
""")


# Ergebnisse ausgeben (printen)
print("### SPARQL-Abfrage-Ergebnisse ###")
print(f"Gefundene Ergebnisse: {len(results)}")
print("-" * 30)

# Ausgabe der Ergebnisse in einer lesbaren Form
for row in results:
    # Die Ergebnisse sind Tupel, die den Variablen in der SELECT-Klausel entsprechen
    # In diesem Fall: ?v, ?prog, ?code
    print(f"Variable (?v):   {row.v}")
    print(f"Programm (?prog): {row.prog}")
    print(f"Code (?code):    {row.code}")
    print("-" * 30)

# Zusätzliche Ausgabe als DataFrame (falls Sie pandas nutzen)
try:
    import pandas as pd
    # Konvertiert die rdflib-Ergebnisse in einen Pandas DataFrame
    df = pd.DataFrame(results, columns=list(results.vars))
    print("\n### Ergebnisse als Pandas DataFrame ###")
    print(df)
except ImportError:
    print("Installieren Sie 'pandas', um die Ergebnisse als DataFrame anzuzeigen: !pip install pandas")

import json

def sparql_results_to_dicts(results):
    """
    Erwartet ein rdflib.Result-Objekt und baut eine Liste von 
    { 'var_uri': ..., 'prog_uri': ..., 'code': ... }.
    """
    rows = []
    for row in results:
        rows.append({
            "var_uri": str(row["v"]),
            "prog_uri": str(row["prog"]),
            "code": str(row["code"]),
        })
    return rows

# Beispiel:
# results = g.query(SPARQL_QUERY)
# rows = sparql_results_to_dicts(results)
# print(json.dumps(rows[0], indent=2, ensure_ascii=False))

### SPARQL-Abfrage-Ergebnisse ###
Gefundene Ergebnisse: 11
------------------------------
Variable (?v):   http://www.semanticweb.org/AgentProgramParams/Var_HRL_ControlOfOutputs
Programm (?prog): http://www.semanticweb.org/AgentProgramParams/Program_HRL_ControlOfOutputs
Code (?code):    import time

def HRL_ControlOfOutputs(F1: bool, A2: bool, CB_Forwards: bool, Start: bool, F4: bool, CB_Backwards: bool, RGB_Forwards01: bool, RGB_Forwards02: bool, A6: bool, Init_RGB_Forwards: bool, RGB_Backwards01: bool, RGB_Backwards02: bool, Init_RGB_Backwards: bool, RGB_VertikalDown01, RGB_VertikalDown02, Init_RGB_VertikalDown, RGB_VertikalUp01, RGB_VertikalUp02, Init_RGB_VertikalUp, RGB_HorizontalForwards01, Init_RGB_HorizontalForwads, RGB_HorizontalBackwards01, RGB_HorizontalBackwards02, Init_RGB_HorizontalBackwards, A1):
    """Auto-generated from PLCopen XML (vereinfachte Semantik)."""

    def OR_1(V_10000000001, V_10000000002):
        result = F1 or A2
        return result
    V_10000000003 

In [4]:
PROGRAM_EXPLAIN_SYSTEM_PROMPT = """
Du bist ein Experte für SPS-Programmierung, PLCopen XML und automatisch generierten Pythoncode.

Du bekommst:
- eine URI einer Variable aus einem Wissensgraphen (var_uri)
- eine URI eines Programms (prog_uri)
- den zugehörigen automatisch generierten Python-Code (code)

Kontext:
- Der Python-Code wurde aus einem FBD-Programm einer SPS (z. B. Beckhoff / TwinCAT) automatisch generiert.
- Der Funktionsname (z. B. HRL_ControlOfOutputs, HRL_OperatingModes, HRL_RGB_AS_HorizontalMoveSensors)
  repräsentiert das ursprüngliche SPS-Programm.
- Die Funktionsparameter entsprechen den Ein- und Ausgangssignalen des SPS-Programms.
- Innere Funktionen wie OR_1, AND_3 usw. repräsentieren logische Verknüpfungen aus FBD.
- Variablennamen, die mit 'V_' beginnen (z. B. V_10000000003), sind interne Zwischenvariablen der Funktionsbausteine
  und sollen NICHT einzeln beschrieben werden. Du darfst sie als "interne Logik" zusammenfassen.
- Besonders wichtig sind:
  - die Namen der Eingangs- und Ausgangssignale,
  - der Programmname,
  - der EXECUTE-Block und sein Kommentar mit dem originalen ST-Code (z. B. CASE state OF ...),
  - alle Signale, die am Ende zurückgegeben werden (z. B. ConveyorBeltForward, D1, A2, RGB_Forwards, ...).

Aufgabe:
- Erstelle eine KURZE technische Beschreibung in deutscher Sprache, was dieses Programm funktional macht.
- Erkläre auf hoher Ebene:
  - Welche Funktion das Programm im Gesamtsystem hat (z. B. Steuerung Förderband, Betriebsartensteuerung, RGB-Achsen etc.).
  - Welche Rolle die verknüpfte Variable (var_uri) ungefähr spielt (z. B. Skill für "MoveForwards", Ausgang für Förderband vorwärts etc.),
    sofern dies aus Namen von var_uri, prog_uri und Code plausibel erkennbar ist.
- Gehe nicht Zeile für Zeile durch, sondern fasse die Logik sinnvoll zusammen.
- Gehe auf die Zustandslogik ein, wenn es einen EXECUTE-Block mit CASE state / jobRunning o. ä. gibt.
- Nenne wichtige Ein- und Ausgänge mit Namen, aber nicht alle internen 'V_...' Variablen.

Format:
Gib deine Antwort bitte in diesem Format aus:

Kurzbeschreibung:
- ein bis drei Sätze zur Gesamtfunktion

Details:
- Stichpunktartige Beschreibung der wichtigsten Ein- und Ausgangssignale
- ggf. Besonderheiten der Zustandslogik (EXECUTE-Block, CASE state, MethodCall_* Signale)

Verwende ausschließlich Informationen, die im Code und in den URIs erkennbar sind.
"""


In [8]:
def describe_program_entry(row, model="llama-3.1-8b-instant"):
    """
    row: Dict mit Schlüsseln 'var_uri', 'prog_uri', 'code'
    Erzeugt eine Kurzbeschreibung für genau dieses Programm.
    """
    var_uri = row["var_uri"]
    prog_uri = row["prog_uri"]
    code = row["code"]

    user_content = f"""
    Erkläre mit bitte, was in diesem Programm gemacht wird.
    var_uri: {var_uri}
    prog_uri: {prog_uri}

    Python-Code:
    ```python
    {code}
    """
    messages = [
        {"role": "system", "content": PROGRAM_EXPLAIN_SYSTEM_PROMPT},
        {"role": "user", "content": user_content},
    ]

    description = call_groq(messages, model=model, temperature=0.0, max_tokens=800)
    return description.strip()

In [9]:
SPARQL_QUERY = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX ag:<http://www.semanticweb.org/AgentProgramParams/>

SELECT * WHERE { 
  ?v a ag:class_Variable.
  ag:Program_HRL_SkillSet ag:op_usesVariable ?v.
  ?v ag:op_isMappedToProgram ?prog.
  ?prog ag:dp_hasProgramCode ?code.
}
"""

results = g.query(SPARQL_QUERY)
rows = sparql_results_to_dicts(results)

program_descriptions = []

for row in rows:
    print("--------------------------------------------------")
    print("Variable:", row["var_uri"])
    print("Programm:", row["prog_uri"])
    print("Code-Ausschnitt (ersten 5 Zeilen):")
    print("\n".join(row["code"].splitlines()[:5]))
    print("\n>>> Generierte Beschreibung:\n")
    desc = describe_program_entry(row)
    print(desc)
    program_descriptions.append({
        "var_uri": row["var_uri"],
        "prog_uri": row["prog_uri"],
        "description": desc,
    })

--------------------------------------------------
Variable: http://www.semanticweb.org/AgentProgramParams/Var_HRL_ControlOfOutputs
Programm: http://www.semanticweb.org/AgentProgramParams/Program_HRL_ControlOfOutputs
Code-Ausschnitt (ersten 5 Zeilen):
import time

def HRL_ControlOfOutputs(F1: bool, A2: bool, CB_Forwards: bool, Start: bool, F4: bool, CB_Backwards: bool, RGB_Forwards01: bool, RGB_Forwards02: bool, A6: bool, Init_RGB_Forwards: bool, RGB_Backwards01: bool, RGB_Backwards02: bool, Init_RGB_Backwards: bool, RGB_VertikalDown01, RGB_VertikalDown02, Init_RGB_VertikalDown, RGB_VertikalUp01, RGB_VertikalUp02, Init_RGB_VertikalUp, RGB_HorizontalForwards01, Init_RGB_HorizontalForwads, RGB_HorizontalBackwards01, RGB_HorizontalBackwards02, Init_RGB_HorizontalBackwards, A1):
    """Auto-generated from PLCopen XML (vereinfachte Semantik)."""


>>> Generierte Beschreibung:

Kurzbeschreibung:
Das Programm `HRL_ControlOfOutputs` steuert verschiedene Ausgänge eines Systems, wie z.B. Förderb

In [ ]:
# Zelle 3: Planungsagent (Klassifikation)

PLAN_SYSTEM_PROMPT = """
Du bist ein Planungsagent in einem Industrie 4.0 Projekt.
Dir wird eine Benutzerfrage gegeben.

Du entscheidest, ob die Frage besser über
1) eine strukturierte SPARQL-Abfrage an den Wissensgraphen (mode = "sparql")
oder
2) eine textbasierte Suche über KG-Inhalte (RAG) (mode = "rag")

beantwortet werden soll.

Regeln:
- Wenn nach konkreten Eigenschaften, Relationen, Pfaden, Variablen, Skills, NodeIds,
  Verknüpfungen oder "wer/wie/wo ist in diesem Graph" gefragt wird → "sparql".
- Wenn eher eine Zusammenfassung von mehreren Fakten, eine Erklärung oder
  "natürlichsprachliche" Sicht auf KG-Inhalte gewünscht ist → "rag".
- Wenn du es nicht eindeutig entscheiden kannst, nimm "rag".

Gib ausschließlich JSON im Format
{"mode": "sparql"}
oder
{"mode": "rag"}
zurück, ohne weiteren Text.
"""

def planning_agent(user_input: str) -> str:
    messages = [
        {"role": "system", "content": PLAN_SYSTEM_PROMPT},
        {"role": "user", "content": user_input},
    ]
    raw = call_groq(messages, temperature=0.0)
    try:
        data = json.loads(raw)
        mode = data.get("mode", "rag")
    except json.JSONDecodeError:
        # Fallback, falls das Modell Mist baut
        mode = "rag"
    if mode not in {"sparql", "rag"}:
        mode = "rag"
    return mode


In [ ]:
# Zelle 4: Text2SPARQL

SPARQL_SYSTEM_PROMPT = """
Du bist ein Assistent, der ausschließlich SPARQL-Queries für einen bekannten Wissensgraphen erzeugt.

Regeln:
- Erzeuge eine einzige gültige SPARQL SELECT-Query.
- Verwende bekannte Prefixes (z.B. : , rdf:, rdfs:, xsd:) so, wie sie in diesem Projekt verwendet werden.
- Die Query soll genau auf die Benutzerfrage zielen.
- Gib KEINE Erklärungen, Kommentare oder Markdown aus, nur den reinen SPARQL-Text.
- Wenn du keine sinnvolle Query erzeugen kannst, gib exakt NO_QUERY aus.
"""

def generate_sparql(user_input: str) -> str:
    messages = [
        {"role": "system", "content": SPARQL_SYSTEM_PROMPT},
        {"role": "user", "content": f"Benutzerfrage: {user_input}"},
    ]
    query = call_groq(messages, temperature=0.0, max_tokens=512)
    return query.strip()

def run_sparql(query: str):
    if query.strip() == "" or query.strip().upper() == "NO_QUERY":
        return []
    try:
        res = g.query(query)
        # rdflib Result in Python-Dicts konvertieren
        results = []
        for row in res:
            row_dict = {}
            for var in res.vars:
                row_dict[str(var)] = str(row[var]) if row[var] is not None else None
            results.append(row_dict)
        return results
    except Exception as e:
        print("SPARQL-Fehler:", e)
        return []


In [ ]:
# Zelle 5: Antwort auf Basis von SPARQL-Ergebnissen erzeugen

SPARQL_ANSWER_SYSTEM_PROMPT = """
Du bist ein Assistent, der NUR auf Basis von SPARQL-Ergebnissen antwortet.
Du bekommst:
- die Originalfrage
- eine Liste von Zeilen (JSON), jede Zeile enthält die Spalten der SPARQL-Ergebnisse.

Regeln:
- Nutze ausschließlich die gelieferten SPARQL-Ergebnisse als Wissen.
- Wenn die Informationen nicht ausreichen, um die Frage sinnvoll zu beantworten,
  gib exakt: keine Ahnung
- Füge nichts hinzu, was nicht explizit aus den Ergebnissen ableitbar ist.
- Antworte kurz und präzise in deutscher Sprache.
"""

def answer_from_sparql_results(user_input: str, results) -> str:
    if not results:
        return "keine Ahnung"

    results_json = json.dumps(results, indent=2, ensure_ascii=False)

    messages = [
        {"role": "system", "content": SPARQL_ANSWER_SYSTEM_PROMPT},
        {"role": "user", "content": f"Frage: {user_input}\n\nSPARQL-Ergebnisse:\n{results_json}"},
    ]
    answer = call_groq(messages, temperature=0.0, max_tokens=512)
    # Sicherheit: falls Modell sich nicht an Regeln hält
    if not answer or "keine Ahnung" in answer.lower() and len(results) == 0:
        return "keine Ahnung"
    return answer.strip()


In [ ]:
# Zelle 6: Einfaches RAG über den KG (String-basierte Suche)

def simple_kg_retrieval(question: str, limit: int = 30):
    """
    Sehr einfache String-Suche in Literalen und Labels.
    Achtung: nicht besonders intelligent, aber 100 % KG-basiert.
    """
    q_str = question.lower().replace('"', "'")

    # Suche in allen Literalen (und optional rdfs:label)
    sparql = f"""
    SELECT ?s ?p ?o WHERE {{
      ?s ?p ?o .
      FILTER(isLiteral(?o) && CONTAINS(LCASE(STR(?o)), "{q_str}"))
    }}
    LIMIT {limit}
    """
    try:
        res = g.query(sparql)
    except Exception as e:
        print("Fehler bei simple_kg_retrieval:", e)
        return []

    triples = []
    for row in res:
        triples.append(
            {
                "s": str(row["s"]),
                "p": str(row["p"]),
                "o": str(row["o"]),
            }
        )
    return triples

RAG_ANSWER_SYSTEM_PROMPT = """
Du bist ein Assistent, der NUR auf Basis der folgenden KG-Tripel antwortet.
Du bekommst:
- die Originalfrage
- eine Liste von Tripeln (Subjekt, Prädikat, Objekt) als JSON.

Regeln:
- Nutze ausschließlich diese Tripel als Wissensquelle.
- Wenn die Tripel die Frage nicht beantworten, gib exakt: keine Ahnung
- Keine externe Weltkenntnis einbringen.
- Antworte kurz und präzise in deutscher Sprache.
"""

def answer_with_rag(user_input: str) -> str:
    triples = simple_kg_retrieval(user_input, limit=30)
    if not triples:
        return "keine Ahnung"

    ctx = json.dumps(triples, indent=2, ensure_ascii=False)
    messages = [
        {"role": "system", "content": RAG_ANSWER_SYSTEM_PROMPT},
        {"role": "user", "content": f"Frage: {user_input}\n\nTripel-Kontext:\n{ctx}"},
    ]
    answer = call_groq(messages, temperature=0.0, max_tokens=512)
    if not answer:
        return "keine Ahnung"
    # Minimale „Safety“ gegen Halluzinationen
    if "keine Ahnung" in answer.lower() and len(triples) == 0:
        return "keine Ahnung"
    return answer.strip()


In [ ]:
# Zelle 7: Chatloop mit Planungsagent

def answer_question(user_input: str) -> str:
    """
    High-Level-Pipeline:
    1. Planungsagent: sparql oder rag
    2. entsprechender Pfad
    3. Fallback: "keine Ahnung"
    """
    mode = planning_agent(user_input)
    print(f"[Planner] Modus: {mode}")

    if mode == "sparql":
        sparql_query = generate_sparql(user_input)
        print("Generierte SPARQL-Query:")
        print(textwrap.indent(sparql_query, "    "))

        results = run_sparql(sparql_query)
        if not results:
            return "keine Ahnung"
        return answer_from_sparql_results(user_input, results)

    elif mode == "rag":
        return answer_with_rag(user_input)

    # Fallback – sollte eigentlich nicht auftreten
    return "keine Ahnung"


def chat():
    print("KG-basierter Chatbot (Eingabe 'exit' zum Beenden)")
    while True:
        user_input = input("Du: ").strip()
        if user_input.lower() in {"exit", "quit"}:
            break
        if not user_input:
            continue
        answer = answer_question(user_input)
        print("Bot:", answer)


# Starten, wenn du möchtest:
# chat()


In [11]:
def stream_groq_answer(messages, model="llama-3.1-8b-instant", temperature=0.0, max_tokens=512):
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_completion_tokens=max_tokens,
        top_p=1,
        stream=True,
    )
    full_text = ""
    for chunk in completion:
        delta = chunk.choices[0].delta
        token = delta.content or ""
        print(token, end="", flush=True)
        full_text += token
    print()
    return full_text.strip()
